In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3080'

In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [16]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [17]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [18]:
# 학습에 사용할 CPU나 GPU를 얻습니다.
device = (
    "cuda" 
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [19]:
# 모델을 정의한다
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
# 손실 함수 및 옵티마이저 정의
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
# 학습하는 함수 정의
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 예측 오류 계산
        pred = model(X) # 예측값을 모델을 통해 얻어옴
        loss = loss_fn(pred, y) # 손실함수를 통해 잔차를 구함
        
        # 오차 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [23]:
# 모델이 학습되고 있는지를 확인하기 위해 테스트 데이터셋으로 성능 확인하는 함수
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        

In [24]:
epochs = 5
for t in range(epochs):
    print(f"Epoch: {t+1}\n-------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch: 1
-------------
loss: 2.311729 [   64/60000]
loss: 2.295801 [ 6464/60000]
loss: 2.269797 [12864/60000]
loss: 2.262243 [19264/60000]
loss: 2.251624 [25664/60000]
loss: 2.221645 [32064/60000]
loss: 2.232239 [38464/60000]
loss: 2.195898 [44864/60000]
loss: 2.196070 [51264/60000]
loss: 2.159328 [57664/60000]
Test Error: 
 Accuracy: 44.3%, Avg loss: 2.152735 

Epoch: 2
-------------
loss: 2.162893 [   64/60000]
loss: 2.153901 [ 6464/60000]
loss: 2.086967 [12864/60000]
loss: 2.106277 [19264/60000]
loss: 2.062526 [25664/60000]
loss: 1.997179 [32064/60000]
loss: 2.032910 [38464/60000]
loss: 1.947770 [44864/60000]
loss: 1.952721 [51264/60000]
loss: 1.886923 [57664/60000]
Test Error: 
 Accuracy: 52.4%, Avg loss: 1.879527 

Epoch: 3
-------------
loss: 1.911430 [   64/60000]
loss: 1.880542 [ 6464/60000]
loss: 1.758065 [12864/60000]
loss: 1.804645 [19264/60000]
loss: 1.703204 [25664/60000]
loss: 1.649122 [32064/60000]
loss: 1.687006 [38464/60000]
loss: 1.583661 [44864/60000]
loss: 1.605145 

In [25]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [26]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [27]:
# 예측
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
